In [9]:
import numpy as np

In [10]:
matriz_conectividad=np.loadtxt("./datos_ej1/matriz_conectividad.dat",unpack=True) #En la matriz de conectividad van en cada fila los nodos conectados (de a pares). Si un elemento conecta el nodo 1 y 2, la fila que representa a ese elemento es 1 2.
matriz_conectividad=np.transpose(matriz_conectividad)
fuerzas_dato, s=np.loadtxt("./datos_ej1/fuerzas_dato_s.dat", unpack=True) #En la primera columna estan las fuerzas que son dato, en la segunda, el nodo de esa fuerza
E,A=np.loadtxt("./datos_ej1/EA.dat",unpack=True) #1era columna: valores de E[N/m^2]. 2da: valores de area (m^2)
(gr_libertad)=np.loadtxt("./datos_ej1/gr_libertad.dat") #1era columna: grados de libertad de la componente x del nodo. 2da columna: grados de libertad de la componente x del nodo
#Valores posibles: 0 ó 1. Si el valor es 0, ese grado de libertad esta restringido. Si es 1, está libre
delta,r=np.loadtxt("./datos_ej1/delta_r.dat",unpack=True) #1era columna: Valores conocidos de desplazamiento. 2da columna: posicion del nodo con valor conocido de desplazamiento
matriz_nodos=np.loadtxt("./datos_ej1/matriz_nodos.dat") #1era columna: coordenada x. #2da columna: coordenada y. Cada fila corresponde a las coordenadas de un nodo
#Ej: Si la fila 2 es "16 0", quiere decir que el nodo 2 tiene posicion 16 en x, 0 en y.
s=s.astype(int)
r=r.astype(int)
print(r)
print(s)

[0 1 3 5 6 7]
[2 4]


In [11]:
n=len(matriz_nodos)
matriz_global=np.zeros((2*n,2*n))
np.set_printoptions(precision = 4, linewidth = 132)
scale=np.max((np.max(matriz_global)))
matriz_global[abs(matriz_global/scale) < 1e-9] = 0

<ipython-input-11-db72cf9a7064>:5: RuntimeWarning: invalid value encountered in true_divide
  matriz_global[abs(matriz_global/scale) < 1e-9] = 0


In [12]:
print(fuerzas_dato,s)

[0. 0.] [2 4]


In [13]:
#Armado de la matriz global
for i in range (len(matriz_conectividad)):
    (a),(b)=matriz_conectividad[i] 
    a=int(a)
    b=int(b)
    x1=matriz_nodos[a,0]
    x2=matriz_nodos[b,0]
    y1=matriz_nodos[a,1]
    y2=matriz_nodos[b,1]
    
    angulo=np.arctan2((y2-y1),(x2-x1)) #angulo entre los elementos y el eje de coordenadas
    #print(angulo*180/np.pi)
    S=np.sin(angulo)
    C=np.cos(angulo)
   # print(S,C)
    L=np.sqrt(((y2-y1)**2)+((x2-x1)**2)) #Largo del elemento
    coef_k=[A[i]*E[i]/L] #coeficientes geometricos
    print(coef_k)
    k=coef_k*np.array([[C**2,C*S,-C**2,-C*S],[C*S,S**2,-C*S,-S**2],[-C**2,-C*S,C**2,C*S],[-C*S,-S**2,C*S,S**2]])
    matriz_global[2*a:2*(a+1), 2*a:2*(a+1)] += k [0:2,0:2]
    matriz_global[2*(a):2*(a+1), 2*(b):2*(b+1)] += k [0:2,2:4]
    matriz_global[2*b:2*(b+1), 2*b:2*(b+1)] += k [2:4,2:4]
    matriz_global[2*b:2*(b+1), 2*a:2*(a+1)] += k [2:4,0:2]
scale=np.max((np.max(matriz_global)))
matriz_global[abs(matriz_global/scale) < 1e-9] = 0  #redondea los valores  menores a 10^-9
print(matriz_global) 
#print(s)

[200000.0]
[200000.0]
[200000.0]
[[ 200000.       0. -200000.       0.       0.       0.       0.       0.]
 [      0.       0.       0.       0.       0.       0.       0.       0.]
 [-200000.       0.  400000.       0. -200000.       0.       0.       0.]
 [      0.       0.       0.       0.       0.       0.       0.       0.]
 [      0.       0. -200000.       0.  400000.       0. -200000.       0.]
 [      0.       0.       0.       0.       0.       0.       0.       0.]
 [      0.       0.       0.       0. -200000.       0.  200000.       0.]
 [      0.       0.       0.       0.       0.       0.       0.       0.]]


In [14]:

k_reducida=matriz_global[np.ix_(s,s)]
k_vinculo=matriz_global[np.ix_(s,r)]
print(k_reducida, k_vinculo)


[[ 400000. -200000.]
 [-200000.  400000.]] [[-200000.       0.       0.       0.       0.       0.]
 [      0.       0.       0.       0. -200000.       0.]]


In [15]:
#Calculo de la matriz k reducida y k vinculo
aux=np.linalg.inv(k_reducida)
f=[]
d=[]
for i in range (len(s)):
    f.append(fuerzas_dato[i])
for i in range (len(r)):
    d.append(delta[i])
u=np.dot(aux,f-(np.dot(k_vinculo,d))) #u es un vector con los desplazamientos que eran incognita
#print(u) 

In [16]:
#armado del vector total de desplazamientos (datos e incognitas)
desplazamientos=np.zeros((2*n))
a=0
for i in (s):
    desplazamientos[i]=u[a]
    a+=1
a=0
for i in ((r)):
    desplazamientos[i]=delta[a]
    a+=1
#print(desplazamientos)

In [17]:
#Calculo de las fuerzas incognitas
aux=matriz_global[np.ix_(r,np.arange(0,2*n))]
a=0 
for i in r:
    F=np.dot(aux,desplazamientos)
   # F[i]=F[i]+fuerzas_dato[a]
    #a+=1
print(F)

[-1333.3333     0.         0.         0.      1333.3333     0.    ]


In [18]:
#Armado del vector total de fuerzas(incognitas y datos)
resultado_fuerzas=np.zeros((2*n))
a=0
for i in r:
    resultado_fuerzas[i]=F[a]
    a+=1
a=0
for i in s:
    resultado_fuerzas[i]=fuerzas_dato[a]
    a+=1


In [19]:
print(resultado_fuerzas)
print(desplazamientos)


[-1333.3333     0.         0.         0.         0.         0.      1333.3333     0.    ]
[0.     0.     0.0067 0.     0.0133 0.     0.02   0.    ]
